# HARPS / ESPRESSO Data Pre-processing for WOBBLE analysis

## Outputs that the code has to create:

1. HDF5 data file that can be read by Wobble for analysis
2. Pipeline RVs from HARPS / ESPRESSO (plot & results)

## Ideas / things that might be nice

1. folded RVs for the TESS TOIs assocatiated with the planet with fit?
2. download the data automatically - at the start?

## Inputs into the function

1. A folder containing the CCF data for the observations

## Steps I do manually currently, and the scripts that are used to do them.

1. Identify missing calibration files
    - get_HARPS_calib.py - script by Meg B? defines a function that checks in the header for the name of the calibration file and creates a text file of the missing files.
    - consider how this might need to change for ESPRESSO
2. Download calibration files identified in 1.
    - use a script by Andy Casey called 'prepare_download' that (I think) logs into the ESO site and gets the download of the files ready, without actually downloading anything yet.
    - in a terminal I then change directory to the data directory and run 'cd data; sh download.sh'
3. Extract all the .tar files
4. Run from_HARPS / from_ESPRESSO (what do these functions actually do?)

In [ ]:
#Main idea: function that takes in the data directory as an argument and does all of the above.

In [1]:
"""
IMPORTING PACKAGES
"""
import wobble
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from tqdm import tqdm
import tarfile
from astropy.io import fits
import re
import time
from bs4 import BeautifulSoup
from astroquery.eso import Eso as ESO

/home/z5345592/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/z5345592/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/z5345592/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/z5345592/anaconda3/envs/py37/lib/python3

In [5]:
def missing_wavelength_files(filelist):
    """identifies missing HARPS calibration files and returns them as a list"""
    missing_files = []
    for f in filelist:
        path = f[0:str.rfind(f,'/')+1]
        sp = fits.open(f)
        header = sp[0].header
        wave_file = header['HIERARCH ESO DRS CAL TH FILE']
        if os.path.isfile(path+wave_file):
            continue
        else:
            missing_files = np.append(missing_files, wave_file)

    return np.unique(missing_files)

In [2]:
#is there a reason why this can't be rolled up with the previous function?

if __name__ == '__main__': #
    filelist = glob.glob('THE CCF FILES IN DATA DIRECTORY')
    np.savetxt('missing_files.txt', missing_files, fmt='%s')

NameError: name 'missing_files' is not defined

In [ ]:
# this is 'prepare_download.py' in its entirety (except for package imports that have been rolled in above:
# note - all other comments below are in the original code - not from Andrew Jolly

def prepare_download(): #set a directory

    cwd = '/home/z5345592/projects/harps_espresso_tess/' 

    ESO_USERNAME = "andrewjolly"

    SKIP = 0 # Skip how many batches at the start? (for if you are re-running this..)
    BATCH = 2000 # How many datasets should we get per ESO request?
    WAIT_TIME = 60 # Seconds between asking ESO if they have prepared our request
    DATA_DIR = os.path.join(cwd, "data")
    MISSING_FILES_PATH = os.path.join(cwd, "missing_files.txt")

    if not os.path.exists(DATA_DIR):
        os.mkdir(DATA_DIR)

    with open(MISSING_FILES_PATH, "r") as fp:
        missing_files = [line.strip() for line in fp.readlines()]

    records = []
    for missing_file in missing_files:
        tar_file = "SAF+{}.tar".format(missing_file.split("_")[0])
        # Check to see if we have this filename already.
        if not os.path.exists(os.path.join(DATA_DIR, tar_file)):
            records.append(tar_file)

    N = len(records)
    I = int(np.ceil(N/float(BATCH)))

    remote_paths = []

    print("In total we will request {} records in {} requests".format(N, I))

    for i in range(I):

        if i < SKIP:
            print("Skipping {}".format(i + 1))
            continue

        print("Starting with batch number {}/{}".format(i + 1, I))

        data = [("dataset", dataset) for dataset in records[i*BATCH:(i + 1)*BATCH]]

        # Login to ESO.
        eso = ESO()
        eso.login(ESO_USERNAME)

        prepare_response = eso._session.request("POST",
            "http://dataportal.eso.org/rh/confirmation", data=data)
        assert prepare_response.ok

        # Additional payload items required for confirmation.
        data += [
            ("requestDescription", ""),
            ("deliveryMediaType", "WEB"), # OR USB_DISK --> Holy shit what the fuck!
            ("requestCommand", "SELECTIVE_HOTFLY"),
            ("submit", "Submit")
        ]

        confirmation_response = eso._session.request("POST", 
            "http://dataportal.eso.org/rh/requests/{}/submission".format(ESO_USERNAME),
            data=data)
        assert confirmation_response.ok

        # Parse the request number so that we can get a download script from ESO later
        _ = re.findall("Request #[0-9]+\w", confirmation_response.text)[0].split()[-1]
        request_number = int(_.lstrip("#"))

        print("Retrieving remote paths for request number {}/{}: {}".format(
            i + 1, I, request_number))

        # Check if ESO is ready for us.
        while True:

            url = "https://dataportal.eso.org/rh/requests/{}".format(ESO_USERNAME)
            check_state = eso._request("GET", url, cache=False)
            root = BeautifulSoup(check_state.text, "html5lib")

            link = root.find(href="/rh/requests/{}/{}".format(
                ESO_USERNAME, request_number))

            image = link.find_next("img")
            state = image.attrs["alt"]

            print("Current state {} on request {} ({}/{})".format(
                state, request_number, i + 1, I))

            if state != "COMPLETE":

                # Remove anything from the astroquery cache.
                for cached_file in glob(os.path.join(eso.cache_location, "*")):
                    os.remove(cached_file)

                print("Sleeping for {} seconds..".format(WAIT_TIME))
                time.sleep(WAIT_TIME)

            else:
                break

        response = eso._request("GET", "{}/{}/script".format(url, request_number))
        
        paths = response.text.split("__EOF__")[-2].split("\n")[1:-2]
        print("Found {} remote paths for request_number {}".format(
            len(paths), request_number))
        remote_paths.extend(paths)

        # Remove anything from the astroquery cache.
        for cached_file in glob(os.path.join(eso.cache_location, "*")):
            os.remove(cached_file)
        
        # We have all the remote paths for this request. At ESO's advice, let's
        # wait another 60 seconds before starting our new request.
        if I > i + 1:
            time.sleep(60)

    # Prepare the script for downloading.
    template_path = os.path.join(cwd, "download.sh.template")
    with open(template_path, "r") as fp:
        contents = fp.read()

    script_path = os.path.join(DATA_DIR, "download.sh")
    with open(script_path, "w") as fp:
        fp.write(contents.replace("$$REMOTE_PATHS$$", "\n".join(remote_paths))\
                        .replace("$$ESO_USERNAME$$", ESO_USERNAME))

    print("Created script {0}".format(script_path))
    print("Now run `cd {}; sh {}` and enter your ESO password when requested."\
        .format(DATA_DIR, os.path.basename(script_path)))

    return

In [ ]:
#need a step here that runs the following command:

#'cd data; sh download.sh'

#question - could I just do this using the last line of the above?

## At this point I have the data and the calibration files in a data directory that has been defined.

# Now need to unzip the TARS 


In [ ]:
def tar_unzipper(data_directory):
    """extracts all tar files in a directory into the same directory"""
    for file in glob.glob(data_directory + '/*.tar'):
        tar = tarfile.open(file, 'r')
        tar.extractall(path = data_directory)
        print('Extracting ' + file + ' to ' + data_directory)

In [ ]:
#need a check here to see if the data is from HARPS or ESPRESSO here.

data = wobble.Data() #create a wobble data object so it can be appended with the information from the CCF files

for filename in tqdm(filenames):
    try:
        sp = wobble.Spectrum()
        sp.from_HARPS(filename, process = True)
        data.append(sp)
    except Exception as e:
        print("File {0} failed; error: {1}".format(filename, e))

In [ ]:
data.write(data_dir + star_name + instrument_name + 'pre_proc.hdf5') #writing the HDF5 file that contains all the info that wobble uses as well as the pipeline RV / dates etc that can create quick plots now

In [14]:
"""Function to get name of star and name of instrument from files - make sure its same star, same instrument?"""

def get_instrument_name(data_directory):
    filelist = glob.glob(data_directory + '*ccf_G2_A.fits')
    file = fits.open(filelist[0])
    header = file[0].header
    instrument_name = (header['INSTRUME'])
    
    return instrument_name  

data_dir = '/home/z5345592/projects/eso_pre_processing/test_data/'
print(get_instrument_name(data_dir))

HARPS


In [15]:
def get_target_name(data_directory):
    filelist = glob.glob(data_directory + '*ccf_G2_A.fits')
    file = fits.open(filelist[0])
    header = file[0].header
    target_name = (header['OBJECT'])
    
    return target_name  

data_dir = '/home/z5345592/projects/eso_pre_processing/test_data/'
print(get_target_name(data_dir))

HD48611
